# Get Psychonaut data
<br>
James Chapman<br>
CIS 830 Advanced Topics in AI – Term Project<br>
Kansas State University<br><br>

This notebook collects URLs from [Psychonaut's Substance Index](https://psychonautwiki.org/wiki/Psychoactive_substance_index).<br>

For each URL, gather webpage text (HTML/BeautifulSoup) ---> create Table/CSV of all drug webpages (wiki style).

For each substance of interest, find the matching substance in the Substance Index.<br>

BONUS: Get experience reports from Psychonaut<br>

TODO: Gather experience reports from EROWID

for example EROWID experience reports<br>
https://www.erowid.org/experiences/subs/exp_Pharms_Tianeptine.shtml<br>
https://erowid.org/experiences/subs/exp_Tizanidine.shtml<br>
https://erowid.org/experiences/subs/exp_TMA-2.shtml<br>
https://erowid.org/experiences/subs/exp_TMA-6.shtml<br>
https://www.erowid.org/experiences/subs/exp_Pharms_Tramadol.shtml<br>
https://www.erowid.org/experiences/subs/exp_U47700.shtml<br>


In [1]:
import requests
import pandas as pd
import numpy as np

from urllib.parse import urljoin
from bs4 import BeautifulSoup, Tag, NavigableString
from fuzzywuzzy import process
from tqdm import tqdm
tqdm.pandas()

# psychonautwiki

Get all substances from psychonautwiki index page.

- Drug name
- URL pointing to drug's wiki

https://psychonautwiki.org/wiki/Psychoactive_substance_index <br>


In [2]:
# Get all substances and links from PsychonautWiki index page
url = "https://psychonautwiki.org/wiki/Psychoactive_substance_index"

response = requests.get(url)
html = response.text

# Parse the HTML
soup = BeautifulSoup(html, "html.parser")

# Locate the main content area (this may vary slightly over time)
content_div = soup.find("div", id="mw-content-text")
all_links = content_div.find_all("a")

drug_links = []
for link in all_links:
    href = link.get("href", "")
    # Filter to include only article links that:
    # • start with "/wiki/"
    # • do not contain ":" (avoiding non-content pages)
    if href.startswith("/wiki/") and ":" not in href:
        drug_name = link.get_text().strip()
        # Exclude empty text and the page’s own title (if needed)
        if drug_name and drug_name.lower() != "psychoactive substance index":
            full_url = "https://m.psychonautwiki.org" + href
            drug_links.append((drug_name, full_url))

# Remove duplicates (if any) and sort alphabetically by drug name.
drug_links = list(set(drug_links))
drug_links.sort(key=lambda x: x[0].lower())

psychonaut_data = pd.DataFrame(drug_links, columns=["drug", "URL"])
# mobile webpage URL?
psychonaut_data['URL'] = psychonaut_data['URL'].str.replace(r'://m\.', '://', regex=True)
psychonaut_data.info(verbose=True)
psychonaut_data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   drug    416 non-null    object
 1   URL     416 non-null    object
dtypes: object(2)
memory usage: 6.6+ KB


,drug,URL
0,"1,4-Butanediol","https://psychonautwiki.org/wiki/1,4-Butanediol"
1,1B-LSD,https://psychonautwiki.org/wiki/1B-LSD
2,1cP-AL-LAD,https://psychonautwiki.org/wiki/1cP-AL-LAD
3,1cP-LSD,https://psychonautwiki.org/wiki/1cP-LSD
4,1cP-MiPLA,https://psychonautwiki.org/wiki/1cP-MiPLA
...,...,...
411,α-PVP,https://psychonautwiki.org/wiki/%CE%91-PVP
412,β-Carbolines,https://psychonautwiki.org/wiki/%CE%92-Carbolines
413,βk-2C-B,https://psychonautwiki.org/wiki/%CE%92k-2C-B
414,γ-Aminobutyric acid,https://psychonautwiki.org/wiki/%CE%93-Aminobu...


In [3]:
substances_of_interest = pd.read_csv('../data/substances_of_interest_list.csv', encoding="utf-8-sig")
substances_of_interest

,drug
0,"2,6-Xylidine"
1,2-amino-5-chloropyridine
2,2-fluoro-2-oxo PCE
3,2-Oxo-3-hydroxy-LSD
4,3-hydroxy flubromazepam
...,...
72,Speciociliatine
73,Temazepam
74,Xylazine
75,Zolpidem


# Match drugs to Dr. McGinty's shared list

(only 28 drugs are on psychonautwiki)

In [4]:
substances_of_interest["psychonaut_name"] = None
substances_of_interest["psychonaut_url"] = None
for index, row in substances_of_interest.iterrows():
    drug = row["drug"]  

    # Find the closest match in the substances_of_interest DataFrame
    match, score = process.extractOne(drug, psychonaut_data["drug"].tolist())
    if score > 98:  ############################### 
        print(match)
        psychonaut_url = psychonaut_data[psychonaut_data["drug"] == match]['URL'].values[0]
        substances_of_interest.at[index, "psychonaut_name"] = match
        substances_of_interest.at[index, "psychonaut_url"] = psychonaut_url

substances_of_interest.info(verbose=True)

Amphetamine
Buprenorphine
CBD
Codeine
Diphenhydramine
Fentanyl
Flubromazepam
Gabapentin
Hydrocodone
Hydromorphone
Ketamine
Lorazepam
LSD
MDA
MDMA
Methadone
Methamphetamine
Morphine
O-Desmethyltramadol
Oxycodone
Oxymorphone
Phencyclidine
Pregabalin
Psilocin
Psilocybin
Temazepam
Zolpidem
Zopiclone
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   drug             77 non-null     object
 1   psychonaut_name  28 non-null     object
 2   psychonaut_url   28 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [5]:
for index, row in substances_of_interest.iterrows():
    if not pd.isna(row["psychonaut_name"]):
        continue
    drug = row["drug"]  
    # Find the top three closest matches 
    top_matches = process.extract(drug, psychonaut_data["drug"].tolist(), limit=3)
    print("------------------------------------------------------------")
    for match, score in top_matches:
        if score > 50:  ################################## 
            print(f"{drug} | {match} | {score}")

#I skimmed through these 2 find a threshold make sure the matches are goo

------------------------------------------------------------
2,6-Xylidine | 2,5-DMA | 86
2,6-Xylidine | 2-AI | 86
2,6-Xylidine | 2-FA | 86
------------------------------------------------------------
2-amino-5-chloropyridine | 2'-Oxo-PCE | 86
2-amino-5-chloropyridine | 2'-Oxo-PCM | 86
2-amino-5-chloropyridine | 2,5-DMA | 86
------------------------------------------------------------
2-fluoro-2-oxo PCE | 2'-Oxo-PCE | 86
2-fluoro-2-oxo PCE | 2'-Oxo-PCM | 86
2-fluoro-2-oxo PCE | 2,5-DMA | 86
------------------------------------------------------------
2-Oxo-3-hydroxy-LSD | LSD | 90
2-Oxo-3-hydroxy-LSD | 1B-LSD | 86
2-Oxo-3-hydroxy-LSD | 1cP-LSD | 86
------------------------------------------------------------
3-hydroxy flubromazepam | Bromazepam | 90
3-hydroxy flubromazepam | Flubromazepam | 90
3-hydroxy flubromazepam | 3,4-CTMP | 86
------------------------------------------------------------
3-hydroxy flubromazepam glucuronide | Bromazepam | 90
3-hydroxy flubromazepam glucuronide | Flu

In [6]:
substances_of_interest['psychonaut_url'] = substances_of_interest['psychonaut_url'].str.replace(r'://m\.', '://', regex=True)
substances_of_interest.to_csv('../data/matching_psychonaut_urls.csv', index=False, encoding="utf-8-sig")
substances_of_interest.info(verbose=True)
substances_of_interest

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   drug             77 non-null     object
 1   psychonaut_name  28 non-null     object
 2   psychonaut_url   28 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


,drug,psychonaut_name,psychonaut_url
0,"2,6-Xylidine",None,None
1,2-amino-5-chloropyridine,None,None
2,2-fluoro-2-oxo PCE,None,None
3,2-Oxo-3-hydroxy-LSD,None,None
4,3-hydroxy flubromazepam,None,None
...,...,...,...
72,Speciociliatine,None,None
73,Temazepam,Temazepam,https://psychonautwiki.org/wiki/Temazepam
74,Xylazine,None,None
75,Zolpidem,Zolpidem,https://psychonautwiki.org/wiki/Zolpidem


In [7]:
pd.unique(substances_of_interest["psychonaut_url"])

array([None, 'https://psychonautwiki.org/wiki/Amphetamine',
       'https://psychonautwiki.org/wiki/Buprenorphine',
       'https://psychonautwiki.org/wiki/CBD',
       'https://psychonautwiki.org/wiki/Codeine',
       'https://psychonautwiki.org/wiki/Diphenhydramine',
       'https://psychonautwiki.org/wiki/Fentanyl',
       'https://psychonautwiki.org/wiki/Flubromazepam',
       'https://psychonautwiki.org/wiki/Gabapentin',
       'https://psychonautwiki.org/wiki/Hydrocodone',
       'https://psychonautwiki.org/wiki/Hydromorphone',
       'https://psychonautwiki.org/wiki/Ketamine',
       'https://psychonautwiki.org/wiki/Lorazepam',
       'https://psychonautwiki.org/wiki/LSD',
       'https://psychonautwiki.org/wiki/MDA',
       'https://psychonautwiki.org/wiki/MDMA',
       'https://psychonautwiki.org/wiki/Methadone',
       'https://psychonautwiki.org/wiki/Methamphetamine',
       'https://psychonautwiki.org/wiki/Morphine',
       'https://psychonautwiki.org/wiki/O-Desmethyltramad

# for each drug website (413 total)

- Scrape entire webpage
- divide webpage content/text into sections
    - get section label (h2, h3, etc.)
    - add text from section

In [8]:
records = []
psychonaut_data = psychonaut_data.rename(columns={'URL':'psychonaut_url'})
for _, row in psychonaut_data.iterrows(): #substances_of_interest.iterrows():
    url   = row['psychonaut_url']
    if pd.isna(url):
        continue

    #######################################################
    # Go to website, get the HTML, and remove unwanted text
    #######################################################
    drug  = row['drug']
    resp  = requests.get(url)
    soup  = BeautifulSoup(resp.text, 'html.parser')

    for tag in soup(['script','style','img']):
        tag.decompose()
    content = soup.find('div', class_='mw-parser-output')
    if not content:
        print(f"Warning: no content found for {url}")
        continue
    
    #######################################################
    # Collect data from the page, organized by section
    #######################################################
    page_data = {'drug': drug, 'url': url}

    # Info box
    infobox = content.find('table', id='InfoTable')
    if infobox:
        # use a pipe to separate cells, strip() to clean up whitespace
        page_data['Infobox'] = infobox.get_text(' | ', strip=True)

    # Summary: capture all text before the first H2 header
    intro_lines = []
    for elt in content.children:
        # stop when we hit the first H2
        if elt.name == 'h2':
            break
        # capture paragraphs, lists, or standalone text
        if elt.name == 'p':
            txt = elt.get_text(' ', strip=True)
            if txt:
                intro_lines.append(txt)
        elif elt.name in ('ul','ol'):
            for li in elt.find_all('li'):
                txt = li.get_text(' ', strip=True)
                if txt:
                    bullet = '- ' + txt if elt.name == 'ul' else f"{li.parent.index(li)+1}. {txt}"
                    intro_lines.append(bullet)
    page_data['Summary'] = "\n".join(intro_lines).strip()

    # for every H2 section
    for header in content.find_all('h2', ''):
        section_name = header.get_text(' ', strip=True)
        lines = []  # will collect lines for this section

        for sib in header.next_siblings:
            if getattr(sib, 'name', None) == 'h2':
                break
            # plain text nodes (e.g. stray strings between tags)
            if isinstance(sib, NavigableString):
                txt = sib.strip()
                if txt:
                    lines.append(txt)

            # sub-headers → new line with the header text
            elif sib.name in ('h3','h4','h5'):
                txt = sib.get_text(' ', strip=True)
                if txt:
                    lines.append(f"\n{txt}")

            # paragraphs → each paragraph on its own line
            elif sib.name == 'p':
                txt = sib.get_text(' ', strip=True)
                if txt:
                    lines.append(txt)

            # unordered lists → each <li> as its own bullet line
            elif sib.name == 'ul':
                for li in sib.find_all('li'):
                    txt = li.get_text(' ', strip=True)
                    if txt:
                        lines.append(f"- {txt}")

            # ordered lists → numbered items
            elif sib.name == 'ol':
                for idx, li in enumerate(sib.find_all('li'), 1):
                    txt = li.get_text(' ', strip=True)
                    if txt:
                        lines.append(f"{idx}. {txt}")

            # tables → serialize header+rows into "Col: Val | Col: Val" lines
            elif sib.name == 'table':
                headers = [th.get_text(strip=True) for th in sib.find_all('th')]
                for tr in sib.find_all('tr'):
                    cells = [td.get_text(strip=True) for td in tr.find_all('td')]
                    if not cells:
                        continue
                    if headers and len(headers) == len(cells):
                        line = " | ".join(f"{h}: {c}" for h, c in zip(headers, cells))
                    else:
                        line = " | ".join(cells)
                    lines.append(line)

            # definition lists → each dt: dd as its own line
            elif sib.name == 'dl':
                terms = sib.find_all('dt')
                defs  = sib.find_all('dd')
                for term, definition in zip(terms, defs):
                    t = term.get_text(' ', strip=True)
                    d = definition.get_text(' ', strip=True)
                    if t and d:
                        lines.append(f"{t}: {d}")
            
            # # join with newline so each entry stays on its own line
        page_data[section_name] = "\n".join(lines).strip()

    # capture panels the same way, if you still need them
    for panel in content.select('div.flex-column.two-columns.very-wide > div.panel.radius'):
        hdr = panel.find(['h2','h3','h4'])
        section_name = hdr.get_text(' ', strip=True) if hdr else 'Panel'
        items = []
        for li in panel.select('ul.featured-table li'):
            txt = li.get_text(' ', strip=True)
            if txt:
                items.append(f"- {txt}")
        page_data[section_name] = "\n".join(items)


    records.append(page_data)

psychonaut_all_pages = pd.DataFrame(records)
psychonaut_all_pages.info(verbose=True, show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 159 columns):
 #    Column                                                       Non-Null Count  Dtype 
---   ------                                                       --------------  ----- 
 0    drug                                                         413 non-null    object
 1    url                                                          413 non-null    object
 2    Infobox                                                      311 non-null    object
 3    Summary                                                      413 non-null    object
 4    Contents                                                     394 non-null    object
 5    Chemistry                                                    347 non-null    object
 6    Pharmacology                                                 337 non-null    object
 7    Subjective effects                                           327 non-null    o

In [9]:
# every column with 9 or fewer non‐nulls
low_cols = [
    col for col in psychonaut_all_pages.columns 
    if psychonaut_all_pages[col].count() <= 9 # # # # # # # # # # # #
]

def merge_rare(row):
    parts = []
    for col in low_cols:
        val = row[col]
        if pd.notna(val) and str(val).strip():
            parts.append(f"{col}:\n{val}")
    return "\n\n".join(parts) if parts else np.nan


# Merge low-count columns into a single "Other" column
psychonaut_all_pages['Other'] = psychonaut_all_pages.apply(merge_rare, axis=1)
psychonaut_all_pages.drop(columns=low_cols, inplace=True)
psychonaut_all_pages.info(verbose=True, show_counts=True)
psychonaut_all_pages = psychonaut_all_pages.drop(['Contents'], axis=1)
psychonaut_all_pages.to_csv('../data/psychonaut_all_pages.csv', index=False, encoding="utf-8-sig")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   drug                         413 non-null    object
 1   url                          413 non-null    object
 2   Infobox                      311 non-null    object
 3   Summary                      413 non-null    object
 4   Contents                     394 non-null    object
 5   Chemistry                    347 non-null    object
 6   Pharmacology                 337 non-null    object
 7   Subjective effects           327 non-null    object
 8   Toxicity and harm potential  329 non-null    object
 9   Legal status                 311 non-null    object
 10  See also                     380 non-null    object
 11  External links               388 non-null    object
 12  References                   390 non-null    object
 13  Physical effects             310 no

In [11]:
stats = []
for col in psychonaut_all_pages.columns:
    s = psychonaut_all_pages[col].dropna().astype(str)
    char_lens = s.str.len()
    word_lens = s.str.split().str.len()
    stats.append({
        'column':     col,
        'char_min':   char_lens.min(),
        'char_max':   char_lens.max(),
        'char_mean':  round(char_lens.mean(), 2),
        'word_min':   word_lens.min(),
        'word_max':   word_lens.max(),
        'word_mean':  round(word_lens.mean(), 2),
    })

# 3. Turn into a DataFrame and inspect
stats_df = pd.DataFrame(stats)
print(stats_df)


                         column  char_min  char_max  char_mean  word_min  \
0                          drug         3        32       9.96         1   
1                           url        35        64      42.71         1   
2                       Infobox       234      2686    1096.45        31   
3                       Summary         0      3877    1203.67         0   
4                     Chemistry         0      6131     770.30         0   
5                  Pharmacology         0      8574    1063.15         0   
6            Subjective effects       133     10858    1321.35        20   
7   Toxicity and harm potential        57     26603    4074.43         6   
8                  Legal status         0      9334    1360.94         0   
9                      See also        17      1235      93.67         2   
10               External links        13      1270     128.75         2   
11                   References       641      1236    1091.95        80   
12          

# BONUS:  get the experience reports

These might be useful. Consumers explained their experiences.

In [20]:
# Extract experience reports from the page content
def extract_experience_reports(soup, base_url):
    reports = []
    header = soup.find(
        lambda tag: tag.name in ['h2', 'h3']
        and 'experience reports' in tag.get_text(strip=True).lower()
    )
    if not header:
        return reports

    for sib in header.next_siblings:
        if not isinstance(sib, Tag):
            continue
        if sib.name in ['h2', 'h3']:
            break

        for a in sib.find_all('a', href=True):
            href = urljoin(base_url, a['href'])
            text = a.get_text(strip=True)

            # skip the general index link
            if 'experience_index' in href.lower() or 'experience index' in text.lower():
                continue

            reports.append([text, href])
    if len(reports) == 0:
        return np.nan
    return reports

# Apply to df
experience_report_urls = psychonaut_data.copy()#substances_of_interest.copy()
experience_lists = []
for _, row in experience_report_urls.iterrows():
    url = row['psychonaut_url']
    if not url:
        experience_lists.append(np.nan)
        continue

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    for tag in soup(['script','style']):
        tag.decompose()

    experience_lists.append(extract_experience_reports(soup, url))

experience_report_urls['experience_reports_urls'] = experience_lists

In [21]:
experience_report_urls.info(verbose=True, show_counts=True)
experience_report_urls.to_csv('../data/experience_report_urls.csv', index=False, encoding="utf-8-sig")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   drug                     416 non-null    object
 1   psychonaut_url           416 non-null    object
 2   experience_reports_urls  396 non-null    object
dtypes: object(3)
memory usage: 9.9+ KB


In [22]:
experience_report_urls

,drug,psychonaut_url,experience_reports_urls
0,"1,4-Butanediol","https://psychonautwiki.org/wiki/1,4-Butanediol","[[Experience:2ml 1,4-Butanediol - Not as good ..."
1,1B-LSD,https://psychonautwiki.org/wiki/1B-LSD,"[[Erowid Experience Vaults: 1B-LSD, https://ww..."
2,1cP-AL-LAD,https://psychonautwiki.org/wiki/1cP-AL-LAD,NaN
3,1cP-LSD,https://psychonautwiki.org/wiki/1cP-LSD,"[[Experience:1050 µg 1cP-LSD - The matrix, htt..."
4,1cP-MiPLA,https://psychonautwiki.org/wiki/1cP-MiPLA,NaN
...,...,...,...
411,α-PVP,https://psychonautwiki.org/wiki/%CE%91-PVP,"[[Erowid Experience Vaults: A-PVP, https://www..."
412,β-Carbolines,https://psychonautwiki.org/wiki/%CE%92-Carbolines,[]
413,βk-2C-B,https://psychonautwiki.org/wiki/%CE%92k-2C-B,"[[Experience:120mg - Garden of The Gods, https..."
414,γ-Aminobutyric acid,https://psychonautwiki.org/wiki/%CE%93-Aminobu...,[]


In [26]:
pairs = []
for _, row in experience_report_urls.iterrows():
    drug = row['drug']
    reports = row.get('experience_reports_urls', [])
    
    if isinstance(reports, list):  # only iterate if it's a list
        for text, url in reports:
            pairs.append({'drug': drug, 'experience_report_url': url})

experience_reports = pd.DataFrame(pairs)
print(experience_reports.to_string(index=False))
experience_reports.info(verbose=True)
# for later

                            drug                                                                                                                                                              experience_report_url
                  1,4-Butanediol                                                                                             https://psychonautwiki.org/wiki/Experience:2ml_1,4-Butanediol_-_Not_as_good_as_alcohol
                  1,4-Butanediol                                                                                                                     https://www.erowid.org/experiences/subs/exp_14Butanediol.shtml
                          1B-LSD                                                                                                                            https://www.erowid.org/experiences/subs/exp_1BLSD.shtml
                         1cP-LSD                                                                                                       https://psychonau

In [27]:

def extract_psycho_report(url):
    """Grab the ‘Report’ section from a PsychonautWiki experience page."""
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    if not content:
        return np.nan

    # Find the header whose text contains "Report"
    header = content.find(
        lambda tag: tag.name in ('h2','h3')
        and 'report' in tag.get_text(strip=True).lower()
    )
    if header is None:
        return np.nan

    parts = [header.get_text(strip=True)]  # include the section title

    # Collect all siblings until the next h2/h3
    for sib in header.next_siblings:
        if isinstance(sib, str):
            continue
        if sib.name in ('h2','h3'):
            break
        if sib.name in ('p','ul','ol','dl'):
            text = sib.get_text(separator='\n', strip=True)
            if text:
                parts.append(text)

    return '\n\n'.join(parts)



experience_reports['experience_reports_text'] = pd.Series(index=experience_reports.index, dtype='object')

# Only follow PsychonautWiki URLs not erowid
mask = experience_reports['experience_report_url'].str.contains('psychonautwiki', na=False)

for idx in experience_reports[mask].index:
    url = experience_reports.at[idx, 'experience_report_url']
    try:
        experience_reports.at[idx, 'experience_reports_text'] = extract_psycho_report(url)
    except Exception as e:
        print(f"Failed on {url}: {e}")

experience_reports.to_csv('../data/experience_reports.csv', index=False, encoding='utf-8-sig')


Failed on https://psychonautwiki.org/w/index.php?title=UserGIGA30&action=edit&redlink=1: 404 Client Error: Not Found for url: https://psychonautwiki.org/wiki/UserGIGA30
Failed on https://psychonautwiki.org/w/index.php?title=User:Sunson_JT&action=edit&redlink=1: 404 Client Error: Not Found for url: https://psychonautwiki.org/wiki/User:Sunson_JT
Failed on https://psychonautwiki.org/w/index.php?title=User:Discogoat&action=edit&redlink=1: 404 Client Error: Not Found for url: https://psychonautwiki.org/wiki/User:Discogoat
Failed on https://psychonautwiki.org/w/index.php?title=Zaleplon&action=edit&redlink=1: 404 Client Error: Not Found for url: https://psychonautwiki.org/wiki/Zaleplon
Failed on https://psychonautwiki.org/w/index.php?title=User:Natzki&action=edit&redlink=1: 404 Client Error: Not Found for url: https://psychonautwiki.org/wiki/User:Natzki
Failed on https://psychonautwiki.org/w/index.php?title=Bupropion&action=edit&redlink=1: 404 Client Error: Not Found for url: https://psychona

In [31]:
experience_reports.info(verbose=True, show_counts=True)
for element in experience_reports['experience_reports_text'][:10]:
    if isinstance(element, str):
        print(f"Length: {len(element)}")
        print(element)
    else:
        print("Not a string")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   drug                     1900 non-null   object
 1   experience_report_url    1900 non-null   object
 2   experience_reports_text  1186 non-null   object
dtypes: object(3)
memory usage: 44.7+ KB
Length: 1363
Report

T+0
- Dosed about 1.6mls, setting: alone at home, third time trying it after two unsuccessful attemts at 1.5ml and 2.5ml (over time).

T+0:30
- I begin to get somewhat dizzy, sort of like the alcohol-feeling, but not as euphoric. It‘s more clearheaded.

T+0:40
- I dose another 0.6ml, as I „don‘t feel that much“; totaling 2.2ml.

T+0:50
- While watching videos on youtube I notice people magically look more beautiful than I would see them normally, no visual distortions or anything. I feel what people are doing generates more emotions for me than I would normally